In [1]:
# fix for keras v3.0 update
import os
#os.environ['TF_USE_LEGACY_KERAS'] = '1' 

import tensorflow as tf
import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.utils import Sequence
from qkeras import *
import keras_tuner as kt

import sys 
parentdir = os.path.dirname(os.getcwd())
sys.path.insert(0, parentdir) 
from FilteringModel import *

2025-06-10 13:06:16.829422: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-10 13:06:16.830635: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-10 13:06:16.833568: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-10 13:06:16.842244: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749578776.856842  409192 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749578776.86

In [2]:
import numpy as np
import OptimizedDataGenerator4 as ODG  # or however you import it

base_dir      = Path("./filtering_records2000")
train_dir     = base_dir / "tfrecords_train"
validation_dir= base_dir / "tfrecords_validation"

# your allowed features
allowed_features = [
    'cluster', 'x_profile', 'y_profile',
    'x_size', 'y_size', 'y_local', 'z_global',
    'total_charge',
    'adjusted_hit_time', 'adjusted_hit_time_30ps_gaussian',
    'adjusted_hit_time_60ps_gaussian'
]

# instantiate generators
train_gen = ODG.OptimizedDataGenerator(
    load_records=True,
    tf_records_dir=str(train_dir),
    x_feature_description=allowed_features
)

val_gen = ODG.OptimizedDataGenerator(
    load_records=True,
    tf_records_dir=str(validation_dir),
    x_feature_description=allowed_features
)

def inspect_generator(gen, name):
    print(f"\n=== Inspecting {name} batch ===")
    # assuming the generator yields (x_dict, y_array)
    x_batch, y_batch = next(iter(gen))
    
    print("Features present:", list(x_batch.keys()))
    # check for any unexpected features
    for feat in x_batch:
        if feat not in allowed_features:
            print(f"  ⚠️  Unexpected feature: {feat}")
    
    # print stats for each feature
    for feat, arr in x_batch.items():
        arr_np = np.asarray(arr)
        print(f"  • {feat:30s} shape={arr_np.shape}, dtype={arr_np.dtype}, "
              f"min={arr_np.min():.3g}, max={arr_np.max():.3g}, "
              f"has_nan={np.isnan(arr_np).any()}")
    
    # inspect labels if needed
    y_np = np.asarray(y_batch)
    print(f"Labels shape: {y_np.shape}, dtype={y_np.dtype}, "
          f"min={y_np.min():.3g}, max={y_np.max():.3g}, "
          f"has_nan={np.isnan(y_np).any()}")

# run inspections
inspect_generator(train_gen, "train")
inspect_generator(val_gen,   "validation")

2025-06-10 13:06:19.758040: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected



=== Inspecting train batch ===
Features present: ['cluster', 'x_profile', 'y_profile', 'x_size', 'y_size', 'y_local', 'z_global', 'total_charge', 'adjusted_hit_time', 'adjusted_hit_time_30ps_gaussian', 'adjusted_hit_time_60ps_gaussian']
  • cluster                        shape=(2000, 13, 21, 1), dtype=float32, min=0, max=14.5, has_nan=False
  • x_profile                      shape=(2000, 21), dtype=float32, min=0, max=95.4, has_nan=False
  • y_profile                      shape=(2000, 13), dtype=float32, min=0, max=155, has_nan=False
  • x_size                         shape=(2000, 1), dtype=float32, min=0.0476, max=0.905, has_nan=False
  • y_size                         shape=(2000, 1), dtype=float32, min=0.0769, max=1, has_nan=False
  • y_local                        shape=(2000, 1), dtype=float32, min=-0.535, max=0.997, has_nan=False
  • z_global                       shape=(2000, 1), dtype=float32, min=-0.999, max=0.992, has_nan=False
  • total_charge                   shape=(2000,

2025-06-10 13:06:19.924536: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:381] TFRecordDataset `buffer_size` is unspecified, default to 262144


Features present: ['cluster', 'x_profile', 'y_profile', 'x_size', 'y_size', 'y_local', 'z_global', 'total_charge', 'adjusted_hit_time', 'adjusted_hit_time_30ps_gaussian', 'adjusted_hit_time_60ps_gaussian']
  • cluster                        shape=(2000, 13, 21, 1), dtype=float32, min=0, max=15, has_nan=False
  • x_profile                      shape=(2000, 21), dtype=float32, min=0, max=120, has_nan=False
  • y_profile                      shape=(2000, 13), dtype=float32, min=0, max=152, has_nan=False
  • x_size                         shape=(2000, 1), dtype=float32, min=0.0476, max=0.952, has_nan=False
  • y_size                         shape=(2000, 1), dtype=float32, min=0.0769, max=1, has_nan=False
  • y_local                        shape=(2000, 1), dtype=float32, min=-0.526, max=1, has_nan=False
  • z_global                       shape=(2000, 1), dtype=float32, min=-0.999, max=0.998, has_nan=False
  • total_charge                   shape=(2000, 1), dtype=float32, min=0.0118, max=0.5

In [3]:
allowed_features = [
    'cluster', 'x_profile', 'y_profile',
    'x_size', 'y_size', 'y_local', 'z_global',
    'total_charge',
    'adjusted_hit_time', 'adjusted_hit_time_30ps_gaussian',
    'adjusted_hit_time_60ps_gaussian'
]

# assume train_gen already defined…
# train_gen = ODG.OptimizedDataGenerator(...)

# Dict to hold the first-seen example for each feature
samples = {}

for x_batch, y_batch in train_gen:
    # x_batch is a dict mapping feature name → array of shape (batch_size, ...)
    for feat in allowed_features:
        if feat not in samples and feat in x_batch:
            arr = np.asarray(x_batch[feat])
            # take the *first* element in the batch:
            samples[feat] = arr[0]
    # once we've seen every feature, stop
    if len(samples) == len(allowed_features):
        break

# now print them out
print("\n=== First-seen samples for each feature ===")
for feat in allowed_features:
    if feat in samples:
        sample = samples[feat]
        print(f"\nFeature: {feat}")
        print(f"  shape = {sample.shape}")
        print(f"  sample value =\n{sample}")
    else:
        print(f"\n⚠️  Never saw feature `{feat}` in the batches you pulled.")


=== First-seen samples for each feature ===

Feature: cluster
  shape = (13, 21, 1)
  sample value =
[[[ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]]

 [[ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]]

 [[ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.       ]
  [ 0.   